<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=0c949e2f1239c3f7de1ba31ee6c4035848b425dcc9a9e228b703bf49bc2ee730
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.1
    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-27 13:46:50
-------------------
qualified stocks: 87
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  400.00
Current:  1.43 C
-------------------
Today PnL: -55.78 K (-0.39%)
Current PnL: -22.72 L (-14.89%)
CY Booked + Current PnL: -8.83 L (-5.79%)
-------------------
Total profit:  1.59 L
Total loss:  -24.31 L
-------------------
Total Booked + Current PnL: 18.15 L (14.45%)
Total Booked PnL: 40.87 L (32.53%)
Curr Year Booked PnL: 13.89 L (9.71%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.31 C
Est FTT PnL: 88.13 L (61.6%)
Deployed:  1.26 C
Current:  1.43 C
CAGR/XIRR %: 7.38%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.00,-0.08,-14.18,16.63,0.09,14383.0,-14291.0,86486.0,100.64,49.0,M-SC,3.54,253.0,-0.99,0.60,13.62,OX40N,NTT,DURABLES
39,INDIGOPNTS,1408.00,-2.54,-9.54,10.56,0.02,16676.0,-16645.0,157914.0,141.07,65.0,M-SC,9.11,234.0,-1.00,1.10,37.36,OX40N,NTT,PAINTS
83,VOLTAS,1530.00,-0.24,8.25,10.57,19.69,21939.0,15813.0,207555.0,-2.32,52.0,X-MC,3.35,78.0,0.72,1.45,15.61,XY25,NTT,AC
43,ITC,452.00,-0.02,-1.73,12.38,10.44,24348.0,-3462.0,196676.0,-41.15,42.0,X-LC,1.03,5.0,-0.14,1.37,3.74,X40,NTT,FMCG
35,ICICIGI,2252.93,-1.60,5.53,13.89,20.19,25552.0,9643.0,183963.0,-17.20,46.0,X-MC,7.31,68.0,0.38,1.29,21.48,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,GILLETTE,11206.78,3.57,3.14,28.39,32.42,74345.0,7974.0,261870.0,-15.11,50.0,X-SC,7.55,85.0,0.11,1.83,18.71,X40,ATH,FMCG
7,ATULAUTO,844.00,2.74,-18.33,78.25,45.57,125975.0,-36142.0,160990.0,3483.78,52.0,M-SC,4.01,244.0,-0.29,1.13,14.14,XY24,NTT,AUTO
58,RAJOOENG,143.10,2.38,-27.37,92.21,39.61,68650.0,-28050.0,74450.0,-50.84,32.0,H-SC,20.68,135.0,-0.41,0.52,4.93,AR,ATH,MISC
67,SIS,528.00,1.99,-21.33,54.14,21.26,47291.0,-23682.0,87350.0,2064.25,59.0,H-SC,3.81,166.0,-0.50,0.61,18.02,OX40N,NTT,MISC
59,RECLTD,446.00,1.85,-1.73,22.87,20.74,45660.0,-3520.0,199650.0,28.81,50.0,M-MC,5.31,189.0,-0.08,1.40,5.01,XY25,NTT,FINANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WHIRLPOOL,2270.0,-11.12,-14.46,112.67,81.92,110643.0,-16597.0,98201.0,-49.86,23.0,M-SC,3.33,236.0,-0.15,0.69,16.25,XR,NTT,DURABLES
44,JCHAC,2282.0,-2.98,-27.05,37.07,-0.01,34562.0,-34571.0,93234.0,18986.96,18.0,M-SC,1.46,233.0,-1.00,0.65,5.57,OX40N,NTT,AC
28,GREENPANEL,537.0,-2.59,-32.95,113.48,43.14,139304.0,-60322.0,122756.0,175.19,31.0,M-SC,5.87,240.0,-0.43,0.86,15.47,XY24,NTT,MISC
39,INDIGOPNTS,1408.0,-2.54,-9.54,10.56,0.02,16676.0,-16645.0,157914.0,141.07,65.0,M-SC,9.11,234.0,-1.00,1.10,37.36,OX40N,NTT,PAINTS
70,SURYODAY,216.0,-1.97,-19.87,53.55,23.03,76835.0,-35588.0,143483.0,55.70,48.0,H-SC,9.45,167.0,-0.46,1.00,42.36,XR,NTT,BANKS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,0.02,0.55,105.58,106.72,130940.0,684.0,124020.0,-50.25,39.0,H-SC,3.09,139.0,0.01,0.87,24.06,AR,ATH,MISC


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,0.40,-4.20,117.02,107.91,165521.0,-6197.0,141447.0,-22.17,38.0,M-SC,10.47,216.0,-0.04,0.99,1.14,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-2.54,-9.54,10.56,0.02,16676.0,-16645.0,157914.0,141.07,65.0,M-SC,9.11,234.0,-1.00,1.10,37.36,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,-0.08,-14.18,16.63,0.09,14383.0,-14291.0,86486.0,100.64,49.0,M-SC,3.54,253.0,-0.99,0.60,13.62,OX40N,NTT,DURABLES
47,KANSAINER,340.00,0.89,-21.20,44.01,13.47,93517.0,-57177.0,212490.0,-67.53,38.0,H-SC,1.47,158.0,-0.61,1.49,8.05,XY24,NTT,PAINTS
67,SIS,528.00,1.99,-21.33,54.14,21.26,47291.0,-23682.0,87350.0,2064.25,59.0,H-SC,3.81,166.0,-0.50,0.61,18.02,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.00,-0.84,1.76,69.29,72.27,119477.0,2982.0,172430.0,-12.89,56.0,M-LC,3.89,99.0,0.02,1.21,8.04,XR,NTT,IT
38,INDIAMART,4810.62,0.02,0.55,105.58,106.72,130940.0,684.0,124020.0,-50.25,39.0,H-SC,3.09,139.0,0.01,0.87,24.06,AR,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.00,-0.84,1.76,69.29,72.27,119477.0,2982.0,172430.0,-12.89,56.0,M-LC,3.89,99.0,0.02,1.21,8.04,XR,NTT,IT
38,INDIAMART,4810.62,0.02,0.55,105.58,106.72,130940.0,684.0,124020.0,-50.25,39.0,H-SC,3.09,139.0,0.01,0.87,24.06,AR,ATH,MISC
86,ZYDUSLIFE,1286.17,-0.52,-1.33,37.46,35.63,77464.0,-2784.0,206790.0,-15.41,41.0,H-MC,3.97,119.0,-0.04,1.45,15.12,AR,ATH,PHARMA
25,FINCABLES,1641.55,0.40,-4.20,117.02,107.91,165521.0,-6197.0,141447.0,-22.17,38.0,M-SC,10.47,216.0,-0.04,0.99,1.14,OX40N,ATH,CABLES
37,IEX,219.00,-0.96,-4.72,55.98,48.63,106655.0,-9431.0,190523.0,-36.71,51.0,H-SC,15.63,136.0,-0.09,1.33,6.29,XR,NTT,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,0.32,-35.27,108.87,35.20,90678.0,-45380.0,83290.0,9.17,26.0,X-SC,13.18,92.0,-0.50,0.58,0.32,X40,NTT,FOOTWEAR
76,TMPV,600.00,-0.35,-15.17,67.60,42.18,158515.0,-41920.0,234490.0,-24.30,28.0,X-LC,1.48,3.0,-0.26,1.64,1.57,XY24,NTT,AUTO
46,JSWINFRA,342.00,0.07,-9.05,26.69,15.23,48633.0,-18131.0,182216.0,-22.11,29.0,X-MC,6.92,66.0,-0.37,1.27,20.28,X40N,NTT,REALTY
55,PGHH,17907.65,0.12,-4.46,40.00,33.76,76746.0,-8955.0,191865.0,-32.64,34.0,X-MC,3.72,57.0,-0.12,1.34,0.51,X40,ATH,FMCG
22,DIXON,18931.72,-1.56,-6.28,29.73,21.58,39046.0,-8802.0,131337.0,-52.72,35.0,X-MC,6.59,56.0,-0.23,0.92,16.27,X40N,ATH,IT


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.0,-0.02,-1.73,12.38,10.44,24348.0,-3462.0,196676.0,-41.15,42.0,X-LC,1.03,5.0,-0.14,1.37,3.74,X40,NTT,FMCG
76,TMPV,600.0,-0.35,-15.17,67.60,42.18,158515.0,-41920.0,234490.0,-24.30,28.0,X-LC,1.48,3.0,-0.26,1.64,1.57,XY24,NTT,AUTO
3,ACC,3906.0,-0.74,-21.39,108.81,64.15,203540.0,-50891.0,187060.0,-54.13,53.0,X-SC,1.49,82.0,-0.25,1.31,4.93,XY24,BTT,CEMENT
36,ICICIPRULI,790.0,0.50,3.61,26.40,30.97,48345.0,6387.0,183125.0,-20.80,58.0,X-MC,1.83,75.0,0.13,1.28,16.63,X40,ATH,INSURANCE
66,SIEMENS,4671.5,-0.82,-11.56,41.93,25.51,69006.0,-21520.0,164575.0,0.93,63.0,H-LC,1.88,49.0,-0.31,1.15,21.28,AR,ATH,ELECTRICAL


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,0.32,-35.27,108.87,35.20,90678.0,-45380.0,83290.0,9.17,26.0,X-SC,13.18,92.0,-0.50,0.58,0.32,X40,NTT,FOOTWEAR
55,PGHH,17907.65,0.12,-4.46,40.00,33.76,76746.0,-8955.0,191865.0,-32.64,34.0,X-MC,3.72,57.0,-0.12,1.34,0.51,X40,ATH,FMCG
9,BAJAJHFL,181.50,0.59,-13.61,71.84,48.45,124439.0,-27289.0,173217.0,-19.87,35.0,X-MC,7.30,64.0,-0.22,1.21,0.63,X40N,ATH,FINANCE
54,PAGEIND,51605.08,-0.26,-4.97,32.66,26.07,50819.0,-8140.0,155600.0,-30.06,38.0,X-MC,9.37,55.0,-0.16,1.09,0.95,X40,ATH,APPARELS
76,TMPV,600.00,-0.35,-15.17,67.60,42.18,158515.0,-41920.0,234490.0,-24.30,28.0,X-LC,1.48,3.0,-0.26,1.64,1.57,XY24,NTT,AUTO


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,QUESS,424.00,-0.62,-28.06,98.55,42.84,46089.0,-18239.0,46767.0,-53.17,41.0,X-SC,37.57,83.0,-0.40,0.33,1.76,XY24,NTT,MISC
60,RELAXO,1176.00,-0.95,-47.09,187.88,52.31,144288.0,-68362.0,76798.0,-43.66,37.0,X-SC,4.45,91.0,-0.47,0.54,2.80,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,0.32,-35.27,108.87,35.20,90678.0,-45380.0,83290.0,9.17,26.0,X-SC,13.18,92.0,-0.50,0.58,0.32,X40,NTT,FOOTWEAR
22,DIXON,18931.72,-1.56,-6.28,29.73,21.58,39046.0,-8802.0,131337.0,-52.72,35.0,X-MC,6.59,56.0,-0.23,0.92,16.27,X40N,ATH,IT
52,MEDANTA,1486.00,-1.47,3.93,19.39,24.08,25583.0,4988.0,131938.0,-4.53,47.0,X-SC,5.57,89.0,0.19,0.92,24.32,XY24,NTT,HEALTHCARE


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4389.97,-1.17,-15.07,40.44,19.28,118823.0,-52131.0,293825.0,-25.89,56.0,X-LC,7.24,1.0,-0.44,2.05,8.62,X40,ATH,IT
41,INFY,2275.00,-0.24,5.80,46.38,54.87,153539.0,18157.0,331045.0,-17.44,60.0,X-LC,2.64,2.0,0.12,2.31,14.57,X40,BTT,IT
76,TMPV,600.00,-0.35,-15.17,67.60,42.18,158515.0,-41920.0,234490.0,-24.30,28.0,X-LC,1.48,3.0,-0.26,1.64,1.57,XY24,NTT,AUTO
82,VBL,671.64,0.11,-5.87,44.13,35.67,131202.0,-18534.0,297308.0,-17.02,54.0,X-LC,5.77,4.0,-0.14,2.08,7.17,X40N,ATH,FMCG
43,ITC,452.00,-0.02,-1.73,12.38,10.44,24348.0,-3462.0,196676.0,-41.15,42.0,X-LC,1.03,5.0,-0.14,1.37,3.74,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,0.27,-37.02,111.02,32.89,53979.0,-28586.0,48621.0,-700.55,64.0,L-MC,5.64,259.0,-0.53,0.34,33.90,XR,NTT,BANKS
6,ASIANTILES,137.00,-0.82,-13.37,109.70,81.67,89154.0,-12539.0,81271.0,7305.56,50.0,L-SC,19.27,271.0,-0.14,0.57,57.38,XR,NTT,CERAMICS
68,SONACOMS,806.63,0.54,-11.78,58.18,39.54,51920.0,-11918.0,89241.0,-31.03,67.0,M-SC,6.60,220.0,-0.23,0.62,25.84,AR,ATH,AUTO
70,SURYODAY,216.00,-1.97,-19.87,53.55,23.03,76835.0,-35588.0,143483.0,55.70,48.0,H-SC,9.45,167.0,-0.46,1.00,42.36,XR,NTT,BANKS
39,INDIGOPNTS,1408.00,-2.54,-9.54,10.56,0.02,16676.0,-16645.0,157914.0,141.07,65.0,M-SC,9.11,234.0,-1.00,1.10,37.36,OX40N,NTT,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,-2.54,-9.54,10.56,0.02,16676.0,-16645.0,157914.0,141.07,65.0,M-SC,9.11,234.0,-1.00,1.10,37.36,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.25,0.03,0.48,20.36,20.95,51507.0,1214.0,252982.0,-0.49,75.0,X-LC,14.54,31.0,0.02,1.77,36.29,X40,ATH,PAINTS
40,INDUSINDBK,1800.00,0.27,-37.02,111.02,32.89,53979.0,-28586.0,48621.0,-700.55,64.0,L-MC,5.64,259.0,-0.53,0.34,33.90,XR,NTT,BANKS
0,5PAISA,593.00,-0.81,-35.07,73.87,12.90,104805.0,-76619.0,141877.0,109.13,58.0,H-SC,13.41,161.0,-0.73,0.99,17.97,OX40N,NTT,FINANCE
79,UNITDSPR,1644.00,-1.12,6.01,13.92,20.77,34551.0,14069.0,248213.0,-3.23,60.0,X-MC,3.75,62.0,0.41,1.73,13.34,X40N,NTT,BREWERIES


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.79
1,25,44.67
2,50,75.58


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.60
MC    29.23
LC    25.19
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.85
X40      21.22
X40N     11.94
XR        9.54
AR        9.10
XY25      9.09
OX40N     7.66
X200      1.87
SR        0.98
MH        0.77
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    23.93
X-MC    22.73
X-LC    19.99
M-SC    11.96
X-SC     8.30
H-MC     4.76
H-LC     3.02
L-SC     1.41
M-MC     1.40
M-LC     1.21
L-LC     0.97
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.79,-4.70,38.56
IT,13.77,-14.85,74.14
FINANCE,9.46,-14.52,62.91
MISC,7.42,-22.47,72.64
ELECTRICAL,5.99,-9.66,49.83
PAINTS,5.97,-8.10,24.16
INSURANCE,4.43,0.23,34.93
PHARMA,3.88,-0.65,32.87
AUTO,3.39,-18.56,69.40


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3052154.0,21
XR,1301400.0,13
AR,1271901.0,10
X40,1001912.0,14
X40N,739519.0,9
OX40N,695253.0,10
XY25,340489.0,6
SR,280158.0,2
MH,74251.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3557857.0,25
M-SC,1348091.0,15
X-MC,1229266.0,16
X-LC,901178.0,11
X-SC,733257.0,8
H-MC,398444.0,3
L-SC,260928.0,3
H-LC,125186.0,2
M-LC,119477.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1216619.0      6
           AR         873641.0      5
           XR         795091.0      7
M-SC       XY24       778211.0      6
X-MC       X40        441198.0      7
X-LC       X40        395691.0      5
X-MC       XY24       363306.0      3
H-SC       OX40N      318097.0      4
X-SC       X40N       293022.0      3
M-SC       OX40N      289308.0      5
H-SC       SR         280158.0      2
X-SC       XY24       275212.0      3
X-MC       X40N       246669.0      4
X-LC       XY24       228817.0      2
H-MC       AR         208455.0      2
X-LC       X40N       199828.0      2
H-MC       XY24       189989.0      1
X-MC       XY25       178093.0      2
L-SC       XR         173080.0      2
X-SC       X40        165023.0      2
M-SC       XR         159773.0      2
           AR         120799.0      2
M-LC       XR         119477.0      1
L-SC       OX40N       87848.0      1
X-LC       XY25        76842.0      2
H-SC       MH          74251.0      1
H-LC       AR          69006.0      1
           X200        56180.0      1
L-MC       XR          53979.0      1
M-MC       XY25        45660.0      1
L-LC       XY25        39894.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 54.0 seconds
